In [1]:
##
## Another Simple Linear Network and the Delta Rule
##

import numpy as np
import matplotlib.pyplot as plt
import numpy.random as R

# supress unnecessary warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

### create input training patterns and training teacher output

Np = # training patterns<br>
nin = # input nodes<br>
nout = # output nodes

In [2]:
#########################################################################################

# +1 -1 +1 -1 -> +1
# +1 +1 +1 +1 -> +1
# +1 +1 +1 -1 -> -1
# +1 -1 -1 +1 -> -1
#
# one soln: W = [-1 -1 +2 +1], B = 0

Np   = 4     # 4 input patterns
nin  = 4     # 4 input nodes (will add 5th bias input when doing it by hand)
nout = 1     # 1 output node

# input patterns (Np x Nin) - note that I am forcing this to have float values
i = np.array([[+1., -1., +1., -1.],
              [+1., +1., +1., +1.],
              [+1., +1., +1., -1.],
              [+1., -1., -1., +1.]]);

# training outputs
t = np.array([[+1.],
              [+1.],
              [-1.],
              [-1.]]);

# note that for this example, we are not going to use any test patterns,
# we'll just train the network and test it on the training patterns

### here we implement delta rule learning by hand

In [3]:
print('********** delta-rule learning by hand **********')

# implementing by hand, we want to add the explicit input (with value 1) via the bias weight
idelta = np.concatenate((i, np.ones((i.shape[0],1))), axis=1)
nindelta = idelta.shape[1]

print(idelta)

********** delta-rule learning by hand **********
[[ 1. -1.  1. -1.  1.]
 [ 1.  1.  1.  1.  1.]
 [ 1.  1.  1. -1.  1.]
 [ 1. -1. -1.  1.  1.]]


In [4]:
# learning rate
LR = .01

# initialize weight matrix to small random values
Wdelta = .10*R.rand(nindelta,nout)-.05;

# number of training epochs
epochs = 5000

# tanh activation function exists in numpy
# np.tanh()

# initialize err array
err = np.zeros((epochs))

# if verbose, print progress
verbose = False

# loop through epochs
for N in range(epochs):
        
    # initalize output
    outdelta = np.zeros((t.shape[0],1))
    
    # shuffle order of patterns (M for remapping) - necessary for sgd
    M = R.permutation(Np)
    
    # incremental learning / stochastic gradient descent
    for p in range(Np):

        # calculate output given input p (using matrix operation in one line of code)
        outdelta[[M[p]],:] = np.tanh(np.matmul(idelta[[M[p]],:], Wdelta))
        
        # now calculate output given input p long-hand (using a for loop instead)
        for J in range(nout):
            net = 0.
            for I in range(nindelta):
                net += idelta[M[p],I] * Wdelta[I,J]
            outdelta[M[p],J] = np.tanh(net)
        
        # calculate error
        err[N] += np.sum((t[M[p],:]-outdelta[M[p],:])**2)
        
        # update weights
        for J in range(nout):               # loop through all output units
            for I in range(nindelta):       # loop through all intput units
                Wdelta[I,J] += LR*(t[M[p],J]-outdelta[M[p],J])*(1-outdelta[M[p],J]**2)*idelta[M[p],I]
                
    # print current err if verbose
    if verbose:
        print(err[N])
        
print('Done training!')

Done training!


In [5]:
# model predictions (testing with the training data here)
outdelta = np.zeros((idelta.shape[0],1))
for p in range(idelta.shape[0]):
    outdelta[[p],:] = np.tanh(np.matmul(idelta[[p],:], Wdelta))
print('Done testing!')

Done testing!


In [6]:
print('*** delta rule by hand ***')

# print performance
for p in range(idelta.shape[0]):
    for I in range(idelta.shape[1]-1):                      # print training pattern
        print('{0:2.0f} '.format(idelta[p,I]), end='') 
    for J in range(nout):
        print('{0:2.0f} '.format(t[p,J]), end='')           # print true answer
    for J in range(nout):
        print('{0:6.3f} '.format(outdelta[p,J]))            # print network output

*** delta rule by hand ***
 1 -1  1 -1  1  0.934 
 1  1  1  1  1  0.934 
 1  1  1 -1 -1 -0.919 
 1 -1 -1  1 -1 -0.940 


In [7]:
print('*** delta rule by hand ***')

# get learned network weights and biases
print('Weights:')
for I in range(idelta.shape[1]-1):
    print('from i{0:d} '.format(I+1), end='')
    for J in range(nout):
        print('{0:7.4f} '.format(Wdelta[I,J]), end='')
    print()
print('Biases:')
for J in range(nout):
    print('{0:7.4f} '.format(Wdelta[-1,J]), end='')

*** delta rule by hand ***
Weights:
from i1 -0.8142 
from i2 -1.6388 
from i3  3.3554 
from i4  1.6388 
Biases:
-0.8487 

### here we implement delta rule learning in Keras

In [8]:
#########################################################################################
##
## define and train neural network - we will discuss details of these keras pieces later
##

print()
print('********** delta-rule learning using Keras **********')


********** delta-rule learning using Keras **********


In [9]:
# import tools for basic keras networks 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

# create architecture of simple neural network model (using tanh activation function here)
network = models.Sequential()
network.add(layers.Dense(nout, 
                         activation='tanh', 
                         input_shape=(nin,)))

# print a model summary
print(network.summary())
print()
for layer in network.layers:
    print('layer name : {} | input shape : {} | output shape : {}'.format(layer.name, layer.input.shape, layer.output.shape))
print()
for layer in network.layers:
    print(layer.get_config())
print()

# configure optimizer
sgd = optimizers.SGD(learning_rate=0.01, decay=1e-6, momentum=0.9)

# compile network
network.compile(optimizer=sgd, 
                loss='mean_squared_error', 
                metrics=['mse'])

C:\Users\kdmen\miniconda3\envs\NSCenv\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kdmen\miniconda3\envs\NSCenv\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kdmen\miniconda3\envs\NSCenv\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kdmen\miniconda3\envs\NSCenv\lib\site-packages\tensorflow\

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 5         
Total params: 5
Trainable params: 5
Non-trainable params: 0
_________________________________________________________________
None

layer name : dense | input shape : (?, 4) | output shape : (?, 1)

{'name': 'dense', 'trainable': True, 'batch_input_shape': (None, 4), 'dtype': 'float32', 'units': 1, 'activation': 'tanh', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None, 'dtype': 'float32'}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {'dtype': 'float32'}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}



In [10]:
# now train the network
history = network.fit(i, 
                      t, 
                      verbose=False, 
                      epochs=5000)
print('Done training!')

Done training!


In [11]:
# model predictions (remember, testing on the training patterns in this example)
out = network.predict(i)
print('Done testing!')

Done testing!


In [12]:
print('*** delta rule in Keras ***')

# print performance
for p in range(i.shape[0]):
    for I in range(i.shape[1]):                         # print training pattern
        print('{0:2.0f} '.format(i[p,I]), end='') 
    for J in range(nout):
        print('{0:2.0f} '.format(t[p,J]), end='')       # print true answer
    for J in range(nout):
        print('{0:6.3f} '.format(out[p,J]))             # print network output

*** delta rule in Keras ***
 1 -1  1 -1  1  0.972 
 1  1  1  1  1  0.972 
 1  1  1 -1 -1 -0.965 
 1 -1 -1  1 -1 -0.974 


In [13]:
print('*** delta rule in Keras ***')

# get learned network weights and biases
W = network.layers[0].get_weights()[0]     # weights input to hidden
B = network.layers[0].get_weights()[1]     # bias to hidden

# get learned network weights and biases
print('Weights:')
for I in range(i.shape[1]):
    print('from i{0:d} '.format(I+1), end='')
    for J in range(nout):
        print('{0:7.4f} '.format(W[I,J]), end='')
    print()
print('Biases:')
for J in range(nout):
    print('{0:7.4f} '.format(B[J]), end='')
print()

*** delta rule in Keras ***
Weights:
from i1 -0.9425 
from i2 -2.0729 
from i3  4.2210 
from i4  2.0729 
Biases:
-1.1537 
